# Transduction from observation to prediction

### Introduction

Excerpt from Wikipedia [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

In logic, statistical inference, and supervised learning, transduction or transductive inference is reasoning from observed, specific (training) cases to specific (test) cases. In contrast, induction is reasoning from observed training cases to general rules, which are then applied to the test cases. The distinction is most interesting in cases where the predictions of the transductive model are not achievable by any inductive model. Note that this is caused by transductive inference on different test sets producing mutually inconsistent predictions.

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction.sh http://josd.github.io/eye/transduction/observation_prediction.sh
! curl -o observation_prediction.py http://josd.github.io/eye/transduction/observation_prediction.py
! curl -o __init__.py http://josd.github.io/eye/transduction/__init__.py
! curl -o sample.observation http://josd.github.io/eye/transduction/sample.observation
! chmod +x observation_prediction.sh
#% rm -fr ~/t2t_train/observation_prediction/transformer-transformer_small/
#% rm -fr ~/t2t_data/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.5/dist-packages (1.6.0)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (0.16.0)
Requirement not upgraded as not directly required: flask in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (0.12.2)
Requirement not upgraded as not directly required: gevent in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (1.2.2)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (1.6.6)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (1.14.2)
Requirement not upgraded as not directly required: requests in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (2.18.4)
Requirement not upgraded as not directly required: gunicorn in /usr/local/lib/python3.5/dist-packages (from 

In [2]:
# See the observation_prediction problem

! pygmentize -g observation_prediction.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPrediction(text_problems.Text2TextProblem):
  """Transduction from observation to prediction."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 50% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 5,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 5,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(10000):
      # wind turbine size factor
      size_f

In [3]:
# See the observation_prediction script

! pygmentize -g observation_prediction.sh

#!/bin/bash
PROBLEM=observation_prediction
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data
TMP_DIR=/tmp/t2t_datagen
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=3 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --wor

In [4]:
# Run the observation_prediction script

! ./observation_prediction.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Importing user module Downloads from path /home/jdroo
[2018-04-25 15:41:02,367] Importing user module Downloads from path /home/jdroo
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction
[2018-04-25 15:41:02,370] Generating problems:
    observation:
      * observation_prediction
INFO:tensorflow:Generating data for observation_prediction.
[2018-04-25 15:41:02,370] Generating data for observation_prediction.
INFO:tensorflow:Found vocab file: /home/jdroo/t2t_data/vocab.observation_prediction.16384.subwords
[2018-04-25 15:41:02,370] Found vocab file: /home/jdroo/t2t_data

INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
[2018-04-25 15:41:13,680] Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
[2018-04-25 15:41:13,687] Building model body
Instructions for updating:
keep_dims is deprecated, use keepdims instead
[2018-04-25 15:41:13,770] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/layers/common_layers.py:553: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
[2018-04-25 15:41:15,765] Transforming body output with symbol_modality_754_256.top
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

[2018-04

INFO:tensorflow:Running local_init_op.
[2018-04-25 15:42:32,183] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-25 15:42:32,230] Done running local_init_op.
INFO:tensorflow:Evaluation [1/3]
[2018-04-25 15:42:33,361] Evaluation [1/3]
INFO:tensorflow:Evaluation [2/3]
[2018-04-25 15:42:33,587] Evaluation [2/3]
INFO:tensorflow:Evaluation [3/3]
[2018-04-25 15:42:33,787] Evaluation [3/3]
INFO:tensorflow:Finished evaluation at 2018-04-25-13:42:33
[2018-04-25 15:42:33,890] Finished evaluation at 2018-04-25-13:42:33
INFO:tensorflow:Saving dict for global step 2900: global_step = 2900, loss = 0.03559239, metrics-observation_prediction/accuracy = 0.9957976, metrics-observation_prediction/accuracy_per_sequence = 0.97414964, metrics-observation_prediction/accuracy_top5 = 0.9989037, metrics-observation_prediction/approx_bleu_score = 0.75451136, metrics-observation_prediction/neg_log_perplexity = -0.09939319, metrics-observation_prediction/rouge_2_fscore = 0.82804865, met

Instructions for updating:
keep_dims is deprecated, use keepdims instead
[2018-04-25 15:42:48,950] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/beam_search.py:93: calling reduce_logsumexp (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Done calling model_fn.
[2018-04-25 15:42:49,039] Done calling model_fn.
INFO:tensorflow:Graph was finalized.
[2018-04-25 15:42:49,241] Graph was finalized.
2018-04-25 15:42:49.241973: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-04-25 15:42:49.350076: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-04-25 15:42:49.350534: I tensorflow/core/

In [5]:
# See the transductions
# For each target the top 3 is shown with their scores (log probability)

! pygmentize -g sample.observation
print("->-")
! pygmentize -g sample.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.34	A_PERSON has BMI class O	-4.27	A_PERSON has BMI class U	-4.95
A_TURBINE producing 9533 kW	-0.35	A_TURBINE producing 16682 kW	-4.03	A_TURBINE producing 2383 kW	-4.12
